# Scene self-test

Cell 1 prints a compact scene summary (paths, counts, bounds, positions).
Cell 2 renders a 3D overlay (buildings + BS + UAV init + UAV altitude band).


In [1]:
import json
from pathlib import Path
import numpy as np
import plotly.io as pio

from notebook_utils import load_buildings_json, print_topk_tallest_buildings, visualize_scene_selftest_from_cfg

BASE = Path("~/Projects/metaRL_merged").expanduser()

# ---- select one config ----
# CFG_PATH = BASE / "inner_loop/model_a/configs/raleigh_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/atlanta_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/boston_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/chapel_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/open_modelA.json"

# CFG_PATH = BASE / "inner_loop/model_a/configs/charlotte_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/chicago_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/durham_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/la_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/seattle_modelA.json"
CFG_PATH = BASE / "inner_loop/model_a/configs/sf_modelA.json"

with open(CFG_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

# ---- resolve key paths ----
PROJECT_ROOT = Path(cfg.get("project_root", str(BASE))).expanduser()
SCENE_XML = (PROJECT_ROOT / cfg["scene_xml"]).resolve()

BUILDINGS_JSON = (PROJECT_ROOT / cfg["buildings"]["boxes_json"]).resolve()

# ---- parse key blocks ----
uav_cfg = cfg["uav"]
bs_cfg = cfg["bs"]
gu_cfg = cfg["gu"]

n_uav = int(uav_cfg.get("n_uav", len(uav_cfg.get("init_xyz", []))))
uav_init_xyz = uav_cfg.get("init_xyz", [])
uav_bounds = uav_cfg.get("bounds", None)
uav_margin = uav_cfg.get("building_margin", None)

bs_positions = bs_cfg.get("positions", [])
n_bs = len(bs_positions)

n_gus = int(gu_cfg.get("num_gus", 0))
gu_xy_region = gu_cfg.get("xy_region", None)
gu_height = gu_cfg.get("height", None)

# ---- load buildings json to get scene extents ----
bd = load_buildings_json(str(BUILDINGS_JSON))
bs = bd.buildings

xs, ys, zs = [], [], []
for b in bs:
    try:
        xs += [float(b["xmin"]), float(b["xmax"])]
        ys += [float(b["ymin"]), float(b["ymax"])]
        zmin = float(b.get("zmin", 0.0))
        zmax = float(b.get("zmax", b.get("roof_zmax", zmin)))
        zs += [zmin, zmax]
    except Exception:
        pass

x_rng = (min(xs), max(xs)) if xs else (None, None)
y_rng = (min(ys), max(ys)) if ys else (None, None)
z_rng = (min(zs), max(zs)) if zs else (None, None)

scene_size = None
if xs and ys and zs:
    scene_size = (x_rng[1] - x_rng[0], y_rng[1] - y_rng[0], z_rng[1] - z_rng[0])

# ---- print compact report ----
print("=== Scene self-test summary ===")
print("config_path:", CFG_PATH)
print("project_root:", PROJECT_ROOT)
print("scene_xml:", SCENE_XML)
print("buildings_json:", BUILDINGS_JSON)
print()

print("--- counts ---")
print("num_buildings:", len(bs))
print("num_uav:", n_uav)
print("num_gus:", n_gus)
print("num_bs:", n_bs)
print()

print("--- extents (from buildings json) ---")
print("x_range:", x_rng)
print("y_range:", y_rng)
print("z_range:", z_rng)
print("scene_size_xyz:", scene_size)
print()

print("--- UAV config ---")
# UAV height range (from bounds z-range)
uav_height_range = None
if uav_bounds is not None and len(uav_bounds) == 3 and len(uav_bounds[2]) == 2:
    uav_height_range = (float(uav_bounds[2][0]), float(uav_bounds[2][1]))

print("uav_bounds:", uav_bounds)
print("uav_height_range:", uav_height_range)
# print("uav_init_xyz (first 5):", uav_init_xyz[:5])
print("building_margin:", uav_margin)
print()

print("--- BS config ---")
print("bs_positions:", bs_positions)
print("aim_mode:", bs_cfg.get("aim_mode", None))
print()

print("--- GU config ---")
print("gu_xy_region:", gu_xy_region)
print("gu_height:", gu_height)
print()

=== Scene self-test summary ===
config_path: /home/jhuang52/Projects/metaRL_merged/inner_loop/model_a/configs/sf_modelA.json
project_root: /home/jhuang52/Projects/metaRL_merged
scene_xml: /data/jhuang52/Projects/metaRL_merged/scenarios/San-Francisco-(SoMa)/final-scene.xml
buildings_json: /data/jhuang52/Projects/metaRL_merged/scenarios/sf_buildings.json

--- counts ---
num_buildings: 2515
num_uav: 5
num_gus: 150
num_bs: 2

--- extents (from buildings json) ---
x_range: (-1062.9154052734375, 1048.40771484375)
y_range: (-967.0293579101562, 921.3159790039062)
z_range: (0.0, 252.38963317871094)
scene_size_xyz: (2111.3231201171875, 1888.3453369140625, 252.38963317871094)

--- UAV config ---
uav_bounds: [[-1000, 1000], [-1000, 1000], [50, 200]]
uav_height_range: (50.0, 200.0)
building_margin: 1.0

--- BS config ---
bs_positions: [[750, -410, 275], [310, -690, 255]]
aim_mode: none

--- GU config ---
gu_xy_region: [[-800, 800], [-800, 800]]
gu_height: 1.5



In [2]:
top5_building_ids, top5_heights = print_topk_tallest_buildings(BUILDINGS_JSON, k=5)

--- Top-K tallest buildings (by height=zmax-zmin) ---
[1] id=element_2062
     height=252.39, zmin=0.00, zmax=252.39
     aabb=([726.4,780.2], [-432.3,-385.9])
     center=[753.2827758789062, -409.09312438964844, 126.19481658935547]
[2] id=element_1572
     height=230.36, zmin=0.00, zmax=230.36
     aabb=([291.3,322.1], [-706.7,-675.7])
     center=[306.72093200683594, -691.2303466796875, 115.17848205566406]
[3] id=element_117
     height=205.00, zmin=0.00, zmax=205.00
     aabb=([796.8,852.1], [-362.3,-308.8])
     center=[824.4457702636719, -335.5635223388672, 102.5]
[4] id=element_2155
     height=201.23, zmin=0.00, zmax=201.23
     aabb=([504.3,550.8], [-503.5,-451.8])
     center=[527.5694580078125, -477.6363525390625, 100.61511993408203]
[5] id=element_1573
     height=200.36, zmin=0.00, zmax=200.36
     aabb=([367.6,398.1], [-714.4,-682.2])
     center=[382.85862731933594, -698.2835998535156, 100.17848205566406]


In [3]:
fig = visualize_scene_selftest_from_cfg(CFG_PATH, highlight_building_ids=top5_building_ids)
fig.show(renderer="iframe")